In [89]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [90]:
lib.utils.set_seed()
train_x, train_y = lib.datasets.get_odd()
network = lib.nets.create_livenet_odd_2(l1=0.1)
# network = lib.nets.LINEAR3(l1=0.1)

In [91]:
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=len(train_x))
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=1000)


LiveNet


In [92]:

trainer.step(1000)


Iˈ0.000 1.300 = 0.403+0.897 lib/trainer.py:89
Iˈ0.787 N0->D0 died at tick 321 lib/livenet.py:191
Iˈ1.250 N1->D0 died at tick 618 lib/livenet.py:191
Iˈ1.250 D0 became dangle, total dangle = 1 lib/death.py:55


In [57]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.0000

In [65]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    val0 = param["params"]["linear1.weight"][0][0].item()
    val1 = param["params"]["linear1.weight"][1][0].item()
    return val1 + val0

def param_picker1(param):
    try:
        val0 = param["params"]["N0->D0"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_picker1)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

Using matplotlib backend: TkAgg


In [72]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.6f}")

D0 -0.624065
D1 0.624068
N0 -1.749485
N1 1.854146
N0->D1 -1.848331
N1->D1 -1.794769
S0->N0 1.744578
S0->N1 -1.850062


In [21]:
param = nn.Parameter(torch.tensor(0.0))
# network.register_parameter("S0->D0", param)
network._parameters.pop("S0->D0")
# network._parameters


KeyError: 'S0->D0'

S1->D1 0.927462
S1->D1 -0.946442
S1->D1 -0.750392
S1->D1 -0.771363
S1->D1 3.772857
